<a href="https://colab.research.google.com/github/lauraaconcari/ADM_HW2/blob/main/RQ7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -P "/content/drive/MyDrive/Colab Notebooks" https://sapienza2023adm.s3.eu-south-1.amazonaws.com/lighter_books.json


--2023-11-02 10:30:36--  https://sapienza2023adm.s3.eu-south-1.amazonaws.com/lighter_books.json
Resolving sapienza2023adm.s3.eu-south-1.amazonaws.com (sapienza2023adm.s3.eu-south-1.amazonaws.com)... 52.95.153.12, 52.95.152.46
Connecting to sapienza2023adm.s3.eu-south-1.amazonaws.com (sapienza2023adm.s3.eu-south-1.amazonaws.com)|52.95.153.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16123393395 (15G) [application/json]
Saving to: ‘/content/drive/MyDrive/Colab Notebooks/lighter_books.json’

lighter_books.json  100%[===================>]  15.02G  29.5MB/s    in 9m 50s  

2023-11-02 10:40:27 (26.1 MB/s) - ‘/content/drive/MyDrive/Colab Notebooks/lighter_books.json’ saved [16123393395/16123393395]



In [2]:
!wget -P "/content/drive/MyDrive/Colab Notebooks" https://sapienza2023adm.s3.eu-south-1.amazonaws.com/lighter_authors.json

--2023-11-02 10:40:27--  https://sapienza2023adm.s3.eu-south-1.amazonaws.com/lighter_authors.json
Resolving sapienza2023adm.s3.eu-south-1.amazonaws.com (sapienza2023adm.s3.eu-south-1.amazonaws.com)... 52.95.151.26, 52.95.151.42
Connecting to sapienza2023adm.s3.eu-south-1.amazonaws.com (sapienza2023adm.s3.eu-south-1.amazonaws.com)|52.95.151.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549013002 (524M) [application/json]
Saving to: ‘/content/drive/MyDrive/Colab Notebooks/lighter_authors.json’

lighter_authors.jso 100%[===================>] 523.58M  22.3MB/s    in 26s     

2023-11-02 10:40:53 (20.4 MB/s) - ‘/content/drive/MyDrive/Colab Notebooks/lighter_authors.json’ saved [549013002/549013002]



In [3]:
filename = "/content/drive/MyDrive/Colab Notebooks/lighter_authors.json"

In [4]:
filename1 = "/content/drive/MyDrive/Colab Notebooks/lighter_books.json"

In [5]:
import pandas as pd

In [6]:
df_chunks = pd.read_json(filename, lines=True, chunksize=10000)
# Create an empty DataFrame to store the results
df = pd.DataFrame()

# Loop through each chunk and append it to the result DataFrame
for chunk in df_chunks:
  df = pd.concat([df, chunk], ignore_index=True)

In [ ]:
df1_chunks = pd.read_json(filename1, lines=True, chunksize=10000)
# Create an empty DataFrame to store the results
df1 = pd.DataFrame()

# Loop through each chunk and append it to the result DataFrame
for chunk in df1_chunks:
  chunk = chunk[['rating_dist']]
  df1 = pd.concat([df1, chunk], ignore_index=True)

In [98]:
df1_chunks = pd.read_json(filename1, lines=True, chunksize=10000)
# Create an empty DataFrame to store the results
df1 = pd.DataFrame()

# Loop through each chunk and append it to the result DataFrame
for chunk in df1_chunks:
  chunk = chunk[['original_publication_date', 'author_id', 'publication_date']]
  df1 = pd.concat([df1, chunk], ignore_index=True)

In [17]:
df1_chunks = pd.read_json(filename1, lines=True, chunksize=10000)
# Create an empty DataFrame to store the results
df1 = pd.DataFrame()

# Loop through each chunk and append it to the result DataFrame
for chunk in df1_chunks:
  chunk = chunk[['id', 'num_pages']]
  df1 = pd.concat([df1, chunk], ignore_index=True)

In [ ]:
#Estimate the probability that a book has over 30% of ratings above 4

#Parse the string to extract the ratings distribution.
def parse_ratings(s):
   ratings = s.split("|")
   ratings_dict = {}
   for rating in ratings:
       if "total" in rating:
           rating, count = rating.split(":")
           ratings_dict[rating] = int(count)
       else:
           rating, count = rating.split(":")
           ratings_dict[int(rating)] = int(count)

   return ratings_dict

#get total ratings and the percentage of ratings above 4
def get_total_ratings(ratings_dict):
   return ratings_dict['total']

def get_percentage_above_4(ratings_dict, total_ratings):
   above_4_sum = sum(count for rating, count in ratings_dict.items() if rating!='total' and rating > 4)
   if total_ratings!=0:
        return (above_4_sum / total_ratings)*100
   else:
        return (0)


def check_probability(ratings_string):
   ratings_dict = parse_ratings(ratings_string)
   total_ratings = get_total_ratings(ratings_dict)
   percentage_above_4 = get_percentage_above_4(ratings_dict, total_ratings)
   return percentage_above_4 >= 30

df1["above_30_percent_4"] = df1["rating_dist"].apply(check_probability)

count=0
for i in df1["above_30_percent_4"]:
  if i==True:
    count+=1

print((count/len(df1["above_30_percent_4"]))*100)

45.015554048129395


In [100]:
#Estimate the probability that an author publishes a new book within two years from its last work.

import pandas as pd
from datetime import datetime

#we want to take in consideration only the first edition of a book as "new book"
df1_filtered = df1[df1['original_publication_date'] == df1['publication_date']]

df1_grouped = df1_filtered.groupby('author_id')['original_publication_date']
author_dict = df1_grouped.apply(list).to_dict()
lessthan2=0
morethan2=0
for author_id, original_publication_date in author_dict.items():

  #formatting and sorting the values in datetime
  try:
    original_publication_date = sorted([datetime.strptime(x, '%Y-%m-%d') for x in original_publication_date if x is not None])
  except ValueError:
    continue #values that cannot be converted will be eliminated

  #calculating time differences
  time_differences = []
  for i in range(len(original_publication_date)-1):
    try:
      diff = original_publication_date[i+1]-original_publication_date[i]
      diff = diff.total_seconds()/(3600*24*365)
      if diff!=0: #we assume you can't publish multiple books in the same day
        time_differences.append(diff)
    except TypeError:
      continue #if the difference can't be done it will not count

  #counting the number of times the difference of publication is less or more than 2 years
  if time_differences: #when it's not an empty list
    for i in time_differences:
      if i<2:
        lessthan2+=1
      else:
        morethan2+=1
  else:
    continue

print(lessthan2/(lessthan2+morethan2)) #the final result

0.7721301078191538


In [ ]:
#In the file list.json, you will find a peculiar list named "The Worst Books of All Time."
#Estimate the probability of a book being included in this list, knowing it has more than 700 pages.
#Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent?
#Explain how you have obtained your answer.

#creating a list with all the ids of the books in the list Worst Books of All Time (wboat)
df3=pd.read_json("list.json", nrows=10, lines=True) #we actually only need the first row
book_ids_wboat=list()
for i in range(len(df3['books'][0])): #df3['books'][0] is the list WBOAT and we don't need nothing else
  book_ids_wboat.append(df3['books'][0][i]['book_id'])
book_ids_wboat = [int(i) for i in book_ids_wboat] #converting to integers

#formatting the 'number of pages' column and filtering the ones with more than 700 pages
import re

df1_modified=df1.dropna(subset=['num_pages'])
df1_modified['num_pages'] = df1_modified['num_pages'].apply(lambda x: int(x) if (isinstance(x, (float, int)) or bool(re.search(r"^\d+$",x))) else None)
df1_filtered=df1_modified[df1_modified['num_pages']>700]

#checking which books are in both the wboat-list and the filtered books dataframe
def is_in_wboat(book_id):
  return(book_id in book_ids_wboat)
df1["wboat_700"]=df1_filtered['id'].apply(is_in_wboat)

#counting the number of books in the intersection and calculating the probability P(X|Y)
count=0
for i in df1["wboat_700"]:
  if i==True:
    count+=1
print(count/len(df1_filtered))

#to answer the last question we count the number of books in the intersection between the wboat-list
#and all the books regardless of the number of pages, then calculating P(X)
df1["wboat"]=df1_modified['id'].apply(is_in_wboat)
count=0
for i in df1["wboat"]:
  if i==True:
    count+=1
print(count/len(df1_modified))
#the two numbers printed represent P(X|Y) and P(X) respectively,
#since they are different it means that the events are NOT Indipendent
